In [1]:
from concrete_autoencoder import ConcreteAutoencoderFeatureSelector
from keras.layers import Dense, Dropout, LeakyReLU
import numpy as np

# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train = np.reshape(x_train, (len(x_train), -1))
# x_test = np.reshape(x_test, (len(x_test), -1))
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)
# print(x_train.shape, y_train.shape)
# print(x_test.shape, y_test.shape)

def decoder(x):
    x = Dense(320)(x)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.1)(x)
    x = Dense(320)(x)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.1)(x)
    x = Dense(1344)(x)
    return x

Using TensorFlow backend.
/home/local/USHERBROOKE/taxc3101/anaconda3/envs/AutoencEnv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/local/USHERBROOKE/taxc3101/anaconda3/envs/AutoencEnv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/local/USHERBROOKE/taxc3101/anaconda3/envs/AutoencEnv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU

sz = 800

model = Sequential()

model.add(Dense(units=sz, input_dim=250))
model.add(LeakyReLU(0.2))
#model.add(Dropout(0.1))
model.add(Dense(units=1000))
model.add(LeakyReLU(0.2))
#model.add(Dropout(0.1))
model.add(Dense(units=1344))

Using TensorFlow backend.
/home/local/USHERBROOKE/taxc3101/anaconda3/envs/AutoencEnv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/local/USHERBROOKE/taxc3101/anaconda3/envs/AutoencEnv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/local/USHERBROOKE/taxc3101/anaconda3/envs/AutoencEnv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

In [2]:
import keras
import tensorflow

# check versions
print(keras.__version__)
print(tensorflow.__version__)

2.1.6
1.7.0


In [ ]:
2.1.6
1.7.0

# MUDI data

In [3]:
import pandas as pd
import os
import keras
import numpy as np
import h5py

In [4]:
class MRISelectorSubjDataset(keras.utils.Sequence):
    """MRI dataset to select features from."""

    def __init__(self, root_dir, dataf, headerf, subj_list, batch_size=100, shuffle=False):
        """
        Args:
            root_dir (string): Directory with the .csv files
            data (string): Data .csv file
            header (string): Header .csv file
            subj_list (list): list of all the subjects to include
        """
        
        self.root_dir = root_dir
        self.dataf = dataf
        self.headerf = headerf
        self.subj_list = subj_list
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        # load the header
        subj = self.subj_list[0]
        self.header = pd.read_csv(os.path.join(self.root_dir,
                                             self.headerf), index_col=0).to_numpy()
        self.ind = self.header[np.isin(self.header[:,1],self.subj_list),0]
#         print(self.ind)
        
        self.indexes = np.arange(len(self.ind)) 

    def __len__(self):
        return int(np.ceil(len(self.ind) / float(self.batch_size)))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
#         print(indexes)

        # Find list of IDs
        list_IDs_temp = [self.ind[k] for k in indexes]
#         print(list_IDs_temp)

        # Generate data
        signals = self.__data_generation(list_IDs_temp)

        return signals, signals

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.ind)) 
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Generate data
        # X = pd.read_csv(os.path.join(self.root_dir, self.dataf), index_col=0, skiprows=lambda x: x not in list_IDs_temp).to_numpy()
        h5f = h5py.File(os.path.join(self.root_dir, self.dataf), 'r')
        X = h5f.get('data1')
        X = X[list_IDs_temp,:]

        return X

In [4]:
class MRIDecoderSubjDataset(keras.utils.Sequence):
    """MRI dataset to select features from."""

    def __init__(self, root_dir, dataf, headerf, subj_list, selecf, input_dim=1344, batch_size=100, shuffle=False):
        """
        Args:
            root_dir (string): Directory with the .csv files
            data (string): Data .csv file
            header (string): Header .csv file
            subj_list (list): list of all the subjects to include
        """
        
        self.root_dir = root_dir
        self.dataf = dataf
        self.headerf = headerf
        self.subj_list = subj_list
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.selecf = selecf
        
        # load the header
        subj = self.subj_list[0]
        self.header = pd.read_csv(os.path.join(self.root_dir,
                                             self.headerf), index_col=0).to_numpy()
        self.ind = self.header[np.isin(self.header[:,1],self.subj_list),0]
#         print(self.ind)
        
        self.indexes = np.arange(len(self.ind)) 
        
        # load the select file
        self.selecind = np.sort(np.loadtxt(self.selecf, dtype=int))
        print(self.selecind)
        
        self.msk = np.logical_not(np.isin(np.arange(input_dim),self.selecind))
#        print(np.arange(input_dim)[self.msk].shape)

    def __len__(self):
        return int(np.ceil(len(self.ind) / float(self.batch_size)))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
#         print(indexes)

        # Find list of IDs
        list_IDs_temp = [self.ind[k] for k in indexes]
#        print(list_IDs_temp)

        # Generate data
        selec_signals, signals = self.__data_generation(list_IDs_temp)
        
#        print(selec_signals.shape)
#        print(signals.shape)

        return selec_signals, signals

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.ind)) 
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Generate data
        # X = pd.read_csv(os.path.join(self.root_dir, self.dataf), index_col=0, skiprows=lambda x: x not in list_IDs_temp).to_numpy()
        h5f = h5py.File(os.path.join(self.root_dir, self.dataf), 'r')
        X = h5f.get('data1')
        X = X[list_IDs_temp,:]
        X1 = X[:,self.selecind]
        # X2 = X[:,self.msk]

        return X1, X

In [5]:
import math
from keras import backend as K
from keras import Model
from keras.layers import Layer, Softmax, Input
from keras.callbacks import EarlyStopping
from keras.initializers import Constant, glorot_normal
from keras.optimizers import Adam
import tensorflow as tf

In [6]:
# import modules to build RunBuilder and RunManager helper classes
from collections  import OrderedDict
from collections import namedtuple
from itertools import product

# Read in the hyper-parameters and return a Run namedtuple containing all the 
# combinations of hyper-parameters
class RunBuilder():
  @staticmethod
  def get_runs(params):

    Run = namedtuple('Run', params.keys())

    runs = []
    for v in product(*params.values()):
      runs.append(Run(*v))
    
    return runs

In [7]:
# put all hyper params into a OrderedDict, easily expandable
params = OrderedDict(
    lr = [.001],
    batch_size = [256]
)

In [8]:
from datetime import datetime

In [9]:
import tensorflow as tf
for run in RunBuilder.get_runs(params):
    logdir = "/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_decoder/runs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S") + f'-{run}' + 'K=250_sz=800_test14_all_nodrop'
    #writer = tf.summary.FileWriter(logdir + "/metrics")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
    monitor_callback = keras.callbacks.ModelCheckpoint('/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_decoder/' + f'{run}' + 'K=250_epoch=500_sz=800_test14_all_nodrop_best.h5', monitor='val_loss', verbose=1, save_best_only=True)
    
    trainset = MRIDecoderSubjDataset(root_dir='/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_AEFS', dataf='data_.hdf5', headerf ='header_.csv',
                                      selecf = '/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_CA_LR/Run(lr=0.001, batch_size=100)K=250_epoch=1000_testnone_unique.txt',
                                      subj_list=np.array([11, 12, 13, 15]), batch_size=run.batch_size)
    
    testset = MRIDecoderSubjDataset(root_dir='/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_AEFS', dataf='data_.hdf5', headerf ='header_.csv',
                                      selecf = '/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_CA_LR/Run(lr=0.001, batch_size=100)K=250_epoch=1000_testnone_unique.txt',
                                      subj_list=np.array([14]), batch_size=run.batch_size)
    
    model.compile(Adam(run.lr), loss = 'mean_squared_error')
    print(model.summary())
    
    model.fit_generator(trainset, epochs=500, callbacks = [tensorboard_callback, monitor_callback], validation_data=testset, workers=8, use_multiprocessing=True)
    
    model.save('/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_decoder/' + f'{run}' + 'K=250_epoch=500_sz=800_test14_all_nodrop.h5')

Instructions for updating:
Use the retry module or similar alternatives.
[   0    9   20   24   28   33   42   43   51   52   54   64   72   82
   96   97  101  108  120  130  131  140  144  148  177  183  184  199
  200  208  216  224  241  252  256  260  264  266  267  272  289  293
  309  328  348  352  359  364  368  372  376  384  388  392  399  404
  406  407  413  428  432  438  439  451  452  463  467  476  492  494
  495  502  516  520  522  523  524  527  533  538  539  543  548  551
  562  563  576  586  588  593  600  604  610  613  619  620  621  643
  644  652  656  658  659  662  663  666  668  671  677  681  688  700
  703  708  712  718  722  723  728  735  744  746  747  755  756  762
  767  772  787  791  792  796  797  812  819  828  835  840  841  848
  852  868  880  884  896  900  904  908  912  916  923  925  930  931
  936  944  948  950  952  954  956  960  968  972  974  979  980  985
  996 1002 1003 1007 1008 1013 1015 1016 1018 1019 1024 1031 1035 1037
 104

In [7]:
from keras.models import load_model
model = load_model('/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_decoder/Run(lr=0.0001, batch_size=256)K=500_epoch=1000_sz=800_test15.h5')

In [11]:
import numpy as np
testset = MRIDecoderSubjDataset(root_dir='/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_AEFS', dataf='data_.hdf5', headerf ='header_.csv',
                                      selecf = '/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_CA_LR/Run(lr=0.001, batch_size=100)K=500_epoch=800_testnone_unique.txt',
                                      subj_list=np.array([15]), batch_size=10000)
preds = model.predict_generator(testset)

[   0    8    9   13   16   20   24   28   33   40   42   43   50   51
   52   54   64   72   81   82   92   96   97  100  101  106  107  108
  112  117  119  120  123  128  130  131  133  140  144  148  152  176
  177  180  182  183  184  185  190  191  196  198  199  200  208  212
  216  220  224  231  236  237  240  241  242  244  252  256  260  261
  264  266  267  268  272  280  288  289  293  296  308  309  312  316
  317  320  321  328  336  344  345  348  352  358  359  364  368  369
  372  376  380  384  388  392  395  398  399  400  403  404  406  407
  408  413  416  420  428  432  436  438  439  444  448  450  451  452
  458  460  462  463  464  465  466  467  476  481  484  488  489  492
  494  495  496  502  504  505  508  509  516  520  522  523  524  526
  527  532  533  538  539  542  543  548  550  551  560  562  563  564
  568  569  572  576  584  585  586  588  593  600  601  602  604  608
  610  611  613  616  618  619  620  621  624  626  627  628  643  644
  649 

In [12]:
preds.shape

(108378, 844)

In [14]:
preds = preds * 20.60488510131836 /1.1032922

In [15]:
from nilearn.masking import apply_mask, unmask
from nilearn import image, plotting

In [17]:
img_file = 'MB_Re_t_moco_registered_applytopup.nii.gz'
msk_file = 'brain_mask.nii.gz'

In [30]:
direc15 = '/mnt/data_imeka/chantal/MUDI/cdmri0015'
data = unmask(np.transpose(preds),mask_img=os.path.join(direc15, msk_file))
data.shape

(78, 90, 56, 844)

In [20]:
import nibabel as nib

In [25]:
img = nib.load(os.path.join(direc15, img_file))
img.header.get_data_shape()

(78, 90, 56, 1344)

In [34]:
nib.save(data, os.path.join(direc15, 'predictions.nii.gz'))

In [40]:
# put all hyper params into a OrderedDict, easily expandable
params = OrderedDict(
    lr = [.001],
    batch_size = [256]
)

In [ ]:
from datetime import datetime

In [ ]:
import tensorflow as tf
for run in RunBuilder.get_runs(params):
    logdir = "/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_decoder/runs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S") + f'-{run}' + 'K=500_sz=800'
    #writer = tf.summary.FileWriter(logdir + "/metrics")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
    
    trainset = MRIDecoderSubjDataset(root_dir='/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_AEFS', dataf='data_.hdf5', headerf ='header_.csv',
                                      selecf = '/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_CA_LR/Run(lr=0.001, batch_size=100)K=500_epoch=800_testnone_unique.txt',
                                      subj_list=np.array([11, 12, 13, 14]), batch_size=run.batch_size)
    
    testset = MRIDecoderSubjDataset(root_dir='/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_AEFS', dataf='data_.hdf5', headerf ='header_.csv',
                                      selecf = '/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_CA_LR/Run(lr=0.001, batch_size=100)K=500_epoch=800_testnone_unique.txt',
                                      subj_list=np.array([15]), batch_size=run.batch_size)
    
    model.compile(Adam(run.lr), loss = 'mean_squared_error')
    print(model.summary())
    
    model.fit_generator(trainset, epochs=1000, callbacks = [tensorboard_callback], validation_data=testset, workers=8, use_multiprocessing=True)
    
    model.save('/home/local/USHERBROOKE/taxc3101/Documents/chantal/MUDI_decoder/' + f'{run}' + 'K=500_epoch=1000_sz=800_test15.h5')